#UE03 - SPARQL Mini-Test

## Preparation
 

In [10]:
# Install required packages
!pip install -q rdflib

# Imports
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace
from rdflib.namespace import FOAF , XSD , RDFS, NamespaceManager 

# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]                  
    rows.append(row)                    #     add the dictionary (row) to the list 
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns     



In [11]:
g = Graph().parse(format='turtle',data='''
  @prefix : <http://example.org/> .
  
:John a :MalePerson; 
  :hasParent :Mary, :Jim2.
:Mary a :FemalePerson; :hasParent :Jane, :Bob . 
:Jim2 a :MalePerson; :hasParent :Jim, :Kate .
:Bob a :MalePerson . 
:Jane a :FemalePerson .
:Jim a :MalePerson .
:Kate a :FemalePerson .
:Bill a :MalePerson; :hasParent :Jane, :Bob . 
''')


print(g.serialize(format='turtle'))

@prefix : <http://example.org/> .

:Bill a :MalePerson ;
    :hasParent :Bob,
        :Jane .

:John a :MalePerson ;
    :hasParent :Jim2,
        :Mary .

:Jim a :MalePerson .

:Jim2 a :MalePerson ;
    :hasParent :Jim,
        :Kate .

:Kate a :FemalePerson .

:Mary a :FemalePerson ;
    :hasParent :Bob,
        :Jane .

:Bob a :MalePerson .

:Jane a :FemalePerson .





## Test A

### Task 1

Select John's grandmother(s). 

In [12]:
df = sparql_select(g,"""
SELECT ?gm
WHERE { 
	:John :hasParent/:hasParent ?gm .
             ?gm a :FemalePerson . 
}""")

df

,gm
0,:Jane
1,:Kate


### Task 2

Select mothers with their number of children. 

In [13]:
df = sparql_select(g,"""
SELECT ?m (COUNT(?c) AS ?noOfChildren)
WHERE { 
	?c :hasParent ?m .
  ?m a :FemalePerson .
}
GROUP BY ?m
""")

df

,m,noOfChildren
0,:Mary,1
1,:Jane,2
2,:Kate,1


## Test B

### Task 1

Retrieve Bill's sister(s), without making a distinction between sisters and half-sisters. 

In [14]:
df = sparql_select(g,"""
SELECT DISTINCT ?sis
WHERE { 
	:Bill :hasParent ?p . 
  ?sis :hasParent ?p ; 
    a :FemalePerson .
}""")

df

,sis
0,:Mary


### Task 2

Get the number of instancer per class.

In [15]:
df = sparql_select(g,"""
SELECT ?c (COUNT(?o) AS ?noOfInstances)
WHERE { 
	?o a ?c.
}
GROUP BY ?c
""")

df

,c,noOfInstances
0,:MalePerson,5
1,:FemalePerson,3
